In [1]:
import tensorflow as tf

2025-04-12 07:09:04.080790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744452544.145445   34562 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744452544.162886   34562 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744452544.291167   34562 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744452544.291239   34562 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744452544.291243   34562 computation_placer.cc:177] computation placer alr

In [2]:
# Verificar se a GPU está disponível
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU está disponível.")
    for gpu in gpus:
        print(f"Nome da GPU: {gpu.name}")
else:
    print("GPU não está disponível.")

GPU está disponível.
Nome da GPU: /physical_device:GPU:0


In [3]:
tf.config.list_logical_devices('GPU')

I0000 00:00:1744452550.670784   34562 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2559 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [4]:
import fireducks.pandas as fd
import pandas as pd
import numpy as np
import time

In [5]:
def generate_data(rows=100000):
    np.random.seed(42)
    base_dates = pd.date_range(start='2020-01-01', periods=3650, freq='D')
    data = {
        'id': np.arange(rows),
        'category': np.random.choice(['A', 'B', 'C', 'D'], size=rows),
        'value': np.random.randint(1, 1000, size=rows),
        'date': np.random.choice(base_dates, size=rows)
    }
    return pd.DataFrame(data)

def benchmark_pandas(df: pd.DataFrame):
    results = {}
    
    start = time.time()
    filtered = df[df['value'] > 500]
    results['filtering_time'] = time.time() - start
    
    start = time.time()
    grouped = df.groupby('category')['value'].sum()
    results['grouping_time'] = time.time() - start
    
    other_df = df[['category', 'date']].drop_duplicates()
    start = time.time()
    joined = df.merge(other_df, on='category')
    results['joining_time'] = time.time() - start
    
    return results

if __name__ == "__main__":
    sizes = [50000]
    for size in sizes:
        print(f"\nBenchmarking com {size} linhas:")
        df = generate_data(size)
        results = benchmark_pandas(df)
        
        print(f"Tempo de filtering: {results['filtering_time']:.4f}s")
        print(f"Tempo de grouping: {results['grouping_time']:.4f}s")
        print(f"Tempo de joining: {results['joining_time']:.4f}s")


Benchmarking com 50000 linhas:
Tempo de filtering: 0.0047s
Tempo de grouping: 0.0058s
Tempo de joining: 20.9955s


In [6]:
def generate_data(rows=100000):
    np.random.seed(42)
    base_dates = fd.date_range(start='2020-01-01', periods=3650, freq='D')
    data = {
        'id': np.arange(rows),
        'category': np.random.choice(['A', 'B', 'C', 'D'], size=rows),
        'value': np.random.randint(1, 1000, size=rows),
        'date': np.random.choice(base_dates, size=rows)
    }
    return fd.DataFrame(data)

def benchmark_pandas(df: fd.DataFrame):
    results = {}
    
    start = time.time()
    filtered = df[df['value'] > 500]
    results['filtering_time'] = time.time() - start
    
    start = time.time()
    grouped = df.groupby('category')['value'].sum()
    results['grouping_time'] = time.time() - start
    
    other_df = df[['category', 'date']].drop_duplicates()
    start = time.time()
    joined = df.merge(other_df, on='category')
    results['joining_time'] = time.time() - start
    
    return results

if __name__ == "__main__":
    sizes = [1000000, 2000000, 50000000]
    for size in sizes:
        print(f"\nBenchmarking com {size} linhas:")
        df = generate_data(size)
        results = benchmark_pandas(df)
        
        print(f"Tempo de filtering: {results['filtering_time']:.4f}s")
        print(f"Tempo de grouping: {results['grouping_time']:.4f}s")
        print(f"Tempo de joining: {results['joining_time']:.4f}s")


Benchmarking com 1000000 linhas:
Tempo de filtering: 0.0014s
Tempo de grouping: 0.0020s
Tempo de joining: 0.0006s

Benchmarking com 2000000 linhas:
Tempo de filtering: 0.0002s
Tempo de grouping: 0.0002s
Tempo de joining: 0.0001s

Benchmarking com 50000000 linhas:
Tempo de filtering: 0.0002s
Tempo de grouping: 0.0003s
Tempo de joining: 0.0001s
